--- Day 13: Point of Incidence ---

With your help, the hot springs team locates an appropriate spring which launches you neatly and precisely up to the edge of Lava Island.

There's just one problem: you don't see any lava.

You do see a lot of ash and igneous rock; there are even what look like gray mountains scattered around. After a while, you make your way to a nearby cluster of mountains only to discover that the valley between them is completely full of large mirrors. Most of the mirrors seem to be aligned in a consistent way; perhaps you should head in that direction?

As you move through the valley of mirrors, you find that several of them have fallen from the large metal frames keeping them in place. The mirrors are extremely flat and shiny, and many of the fallen mirrors have lodged into the ash at strange angles. Because the terrain is all one color, it's hard to tell where it's safe to walk or where you're about to run into a mirror.

You note down the patterns of ash (.) and rocks (#) that you see as you walk (your puzzle input); perhaps by carefully analyzing these patterns, you can figure out where the mirrors are!

For example:

```
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#
```

To find the reflection in each pattern, you need to find a perfect reflection across either a horizontal line between two rows or across a vertical line between two columns.

In the first pattern, the reflection is across a vertical line between two columns; arrows on each of the two columns point at the line between the columns:

```
123456789
    ><   
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.
    ><   
123456789
```

In this pattern, the line of reflection is the vertical line between columns 5 and 6. Because the vertical line is not perfectly in the middle of the pattern, part of the pattern (column 1) has nowhere to reflect onto and can be ignored; every other column has a reflected column within the pattern and must match exactly: column 2 matches column 9, column 3 matches 8, 4 matches 7, and 5 matches 6.

The second pattern reflects across a horizontal line instead:

```
1 #...##..# 1
2 #....#..# 2
3 ..##..### 3
4v#####.##.v4
5^#####.##.^5
6 ..##..### 6
7 #....#..# 7
```

This pattern reflects across the horizontal line between rows 4 and 5. Row 1 would reflect with a hypothetical row 8, but since that's not in the pattern, row 1 doesn't need to match anything. The remaining rows match: row 2 matches row 7, row 3 matches row 6, and row 4 matches row 5.

To summarize your pattern notes, add up the number of columns to the left of each vertical line of reflection; to that, also add 100 multiplied by the number of rows above each horizontal line of reflection. In the above example, the first pattern's vertical line has 5 columns to its left and the second pattern's horizontal line has 4 rows above it, a total of 405.

Find the line of reflection in each of the patterns in your notes. What number do you get after summarizing all of your notes?

In [ ]:
from math import ceil, floor
import re
import numpy as np

def read_input(filePath):
    with open(filePath, 'r') as file:
        return file.read()

def read_patterns(text):
    patterns = re.findall(r'.*?\n\n', text, re.DOTALL)
    return [[[char for char in line] for line in pattern.strip().splitlines()] for pattern in patterns]

#
#   012345678
#       ><   
#   #.##..##.
#   ..#.##.#. 
#
def has_symmetry_in_x(pattern, candidate):
    width = len(pattern[0])
    steps = min(ceil(candidate), floor(width - candidate))
    for dx in np.arange(0.5, 0.5 + steps, 1):
        positivex = int(candidate + dx)
        negativex = int(candidate - dx)
        for line in pattern:
            if line[positivex] != line[negativex]:
                return False
    return True

def find_col_symmetries(pattern):
    symmetries = []
    width = len(pattern[0])
    for candidate in np.arange(0.5, width - 0.5, 1):
        if has_symmetry_in_x(pattern, candidate):
            symmetries.append(floor(candidate + 1)) # to one-based index
    return symmetries

def find_row_symmetries(pattern):
    return find_col_symmetries(np.transpose(pattern))

def print_pattern(pattern):
    for line in pattern:
        print(''.join(line))

input = read_input('sample.txt')
patterns = read_patterns(input)

# print(find_col_symmetry(patterns[0]))
# print(find_row_symmetry(patterns[1]))
# print_pattern(patterns[0])

answer = 0
for (index, pattern) in enumerate(patterns):
    col_symmetries = find_row_symmetries(pattern)
    col_symmetries = find_col_symmetries(pattern)
    if col_symmetries:
        print(f'found row symmetry for pattern {index} at {col_symmetries[0]}.')
        answer += 100 * col_symmetries[0]
    if col_symmetries:
        print(f'found col symmetry for pattern {index} at {col_symmetries[0]}.')
        answer += col_symmetries[0]
    
print('----------------')
print(answer)

--- Part Two ---

You resume walking through the valley of mirrors and - SMACK! - run directly into one. Hopefully nobody was watching, because that must have been pretty embarrassing.

Upon closer inspection, you discover that every mirror has exactly one smudge: exactly one . or # should be the opposite type.

In each pattern, you'll need to locate and fix the smudge that causes a different reflection line to be valid. (The old reflection line won't necessarily continue being valid after the smudge is fixed.)

Here's the above example again:

```
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#
```

The first pattern's smudge is in the top-left corner. If the top-left # were instead ., it would have a different, horizontal line of reflection:

```
1 ..##..##. 1
2 ..#.##.#. 2
3v##......#v3
4^##......#^4
5 ..#.##.#. 5
6 ..##..##. 6
7 #.#.##.#. 7
```

With the smudge in the top-left corner repaired, a new horizontal line of reflection between rows 3 and 4 now exists. Row 7 has no corresponding reflected row and can be ignored, but every other row matches exactly: row 1 matches row 6, row 2 matches row 5, and row 3 matches row 4.

In the second pattern, the smudge can be fixed by changing the fifth symbol on row 2 from . to #:

```
1v#...##..#v1
2^#...##..#^2
3 ..##..### 3
4 #####.##. 4
5 #####.##. 5
6 ..##..### 6
7 #....#..# 7
```

Now, the pattern has a different horizontal line of reflection between rows 1 and 2.

Summarize your notes as before, but instead use the new different reflection lines. In this example, the first pattern's new horizontal line has 3 rows above it and the second pattern's new horizontal line has 1 row above it, summarizing to the value 400.

In each pattern, fix the smudge and find the different line of reflection. What number do you get after summarizing the new reflection line in each pattern in your notes?

In [ ]:
import copy

def generate_new_patterns(pattern):
    for (row, line) in enumerate(pattern):
        for (col, char) in enumerate(line):
            new_pattern = copy.deepcopy(pattern)
            new_pattern[row][col] = '#' if char == '.' else '.'
            yield new_pattern

def find_new_row_symmetries(pattern):
    return find_new_col_symmetries(np.transpose(pattern))

def find_new_col_symmetries(pattern):
    col_symmetries = find_col_symmetries(pattern)
    new_col_symmetries = []
    for new_pattern in generate_new_patterns(pattern):
        all_col_symmetries = find_col_symmetries(new_pattern)
        new_col_symmetries.extend([s for s in all_col_symmetries if s not in col_symmetries])
    return new_col_symmetries
    
input = read_input('sample.txt')
patterns = read_patterns(input)

answer = 0
for (index, pattern) in enumerate(patterns):
    row_symmetries = find_new_row_symmetries(pattern)
    if row_symmetries:
        print(f'found new row symmetry for pattern {index} at {row_symmetries[0]}.')
        answer += 100 * floor(row_symmetries[0])
    col_symmetries = find_new_col_symmetries(pattern)
    if col_symmetries:
        print(f'found new col symmetry for pattern {index} at {col_symmetries[0]}.')
        answer += floor(col_symmetries[0])

print('----------------')
print(answer)